#KazParC processing

In [ ]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -U "huggingface_hub[cli]" -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

kazparc_ds = load_dataset("issai/kazparc", "kazparc")
kazparc_ds

Generating train split:   0%|          | 0/1742956 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/435742 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/28500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'source_lang', 'target_lang', 'domain', 'pair'],
        num_rows: 1742956
    })
    validation: Dataset({
        features: ['id', 'source_lang', 'target_lang', 'domain', 'pair'],
        num_rows: 435742
    })
    test: Dataset({
        features: ['id', 'source_lang', 'target_lang', 'domain', 'pair'],
        num_rows: 28500
    })
})

In [ ]:
kazparc_df = pd.DataFrame()
for ds in kazparc_ds.values():
    kazparc_df = pd.concat([kazparc_df, ds.to_pandas()])
kazparc = kazparc_df.loc[kazparc_df['pair'] == 'en_ru']
kazparc.reset_index(inplace=True, drop=True)
kazparc

,id,source_lang,target_lang,domain,pair
0,ES017460ES,"To demonstrate his invention, he cycled 60 mil...","Чтобы продемонстрировать своё изобретение, он ...",edu_and_sci,en_ru
1,MM000823MM,The main value of the Constitution is the form...,Главной ценностью Конституции является формиро...,mass_media,en_ru
2,GN027390GN,A zipped up suitcase sitting upright on the gr...,Застегнутый чемодан расположен вертикально на ...,general,en_ru
3,ES043881ES,"We share our stories, our songs, our history, ...","Мы делимся нашими историями, нашими песнями, н...",edu_and_sci,en_ru
4,LD000985LD,The notification shall contain sufficient info...,Подобное уведомление должно содержать информац...,legal_docs,en_ru
...,...,...,...,...,...
368201,GN065597GN,Be careful not to come in laughing and come ou...,Бойся весело входящего и хмуро уходящего.,general,en_ru
368202,ES019365ES,"Hence, it is of no surprise that the most popu...","Следовательно, неудивительно, что наиболее поп...",edu_and_sci,en_ru
368203,MM014332MM,An ample supply of sound information is not su...,Обильного предложения достоверной информации н...,mass_media,en_ru
368204,MM061621MM,The distribution of rental housing has already...,С мая т. г. уже начали выдавать арендное жилье.,mass_media,en_ru


In [ ]:
kazparc = kazparc.drop(columns=['id', 'pair'])
kazparc.head()

,source_lang,target_lang,domain
0,"To demonstrate his invention, he cycled 60 mil...","Чтобы продемонстрировать своё изобретение, он ...",edu_and_sci
1,The main value of the Constitution is the form...,Главной ценностью Конституции является формиро...,mass_media
2,A zipped up suitcase sitting upright on the gr...,Застегнутый чемодан расположен вертикально на ...,general
3,"We share our stories, our songs, our history, ...","Мы делимся нашими историями, нашими песнями, н...",edu_and_sci
4,The notification shall contain sufficient info...,Подобное уведомление должно содержать информац...,legal_docs


In [ ]:
with open('/content/drive/MyDrive/data/idioms/idioms.txt', 'r', encoding='utf-8') as ifile:
    idioms_list = ifile.read().split('\n')
idioms_list = list(set(idioms_list))
print(len(idioms_list), idioms_list[:10])

1596 ['have a bone to pick with someone', 'hot to trot', 'get your skates on', 'fine feathers', 'like death warmed up', "that's the way the cookie crumbles", 'on spec', "there's many a good tune played on an old fiddle", 'twist in the wind', 'fight a losing battle']


In [ ]:
def lemmatize(text: str) -> str:
    text_lem = []
    for word, pos in pos_tag(word_tokenize(text)):
        word = word.lower()
        pos = pos.lower()
        if pos == 'dt':
            continue
        if 'prp' in pos or word in ('something', "one", 'someone'):
            text_lem.append(f'<{pos[0]}>')
        elif pos[0] in ('a', 'n', 'v', 'r'):
            text_lem.append(wnl.lemmatize(word, pos[0]))
        else:
            text_lem.append(wnl.lemmatize(word))
    return ' '.join(text_lem)

In [ ]:
wnl = WordNetLemmatizer()
idioms_lemmatized = []
for idiom in idioms_list:
    idioms_lemmatized.append(lemmatize(idiom))
idioms_lemmatized[:10]

['have bone to pick with <n>',
 'hot to trot',
 'get <p> skate on',
 'fine feather',
 'like death warm up',
 "'s way cookie crumbles",
 'on spec',
 "there 's many good tune play on old fiddle",
 'twist in wind',
 'fight losing battle']

In [ ]:
def detect_idioms(text):
    text_lem = lemmatize(text).split()
    detected_idioms = []
    for idid, idiom in enumerate(idioms_lemmatized):
        search_text = text_lem.copy()
        matches = 0
        for token in idiom.split():
            if token not in search_text:
                break
            matches += 1
            search_text.remove(token)
        if matches == len(idiom.split()):
            detected_idioms.append(idioms_list[idid])
    if detected_idioms:
        return '; '.join(detected_idioms)
    return pd.NA

In [ ]:
from tqdm import tqdm

tqdm.pandas()

In [ ]:
kazparc['idiom'] = kazparc['source_lang'].progress_apply(detect_idioms)

100%|██████████| 368206/368206 [21:56<00:00, 279.64it/s]


In [ ]:
kazparc.head()

,source_lang,target_lang,domain,idiom
0,"To demonstrate his invention, he cycled 60 mil...","Чтобы продемонстрировать своё изобретение, он ...",edu_and_sci,<NA>
1,The main value of the Constitution is the form...,Главной ценностью Конституции является формиро...,mass_media,<NA>
2,A zipped up suitcase sitting upright on the gr...,Застегнутый чемодан расположен вертикально на ...,general,<NA>
3,"We share our stories, our songs, our history, ...","Мы делимся нашими историями, нашими песнями, н...",edu_and_sci,<NA>
4,The notification shall contain sufficient info...,Подобное уведомление должно содержать информац...,legal_docs,<NA>


In [ ]:
len(kazparc.dropna())

11024

In [ ]:
kazparc.dropna()

,source_lang,target_lang,domain,idiom
32,"The observers of political parties, other publ...","Наблюдатели политических партий, иных обществе...",legal_docs,on the take
41,But one of the most interesting facts of my bi...,Но одним из самых интересных фактов биографии ...,mass_media,the facts of life
98,A white dinner plate with some food still on it.,"Белая обеденная тарелка, на которой еще остава...",general,on your plate
124,"To erect barriers to innovation, such as taxes...","Возводить барьеры на пути инноваций, такие как...",mass_media,every which way
154,The provisions of part one of this Paragraph s...,Положения части первой настоящего пункта приме...,legal_docs,on terms
...,...,...,...,...
368101,Time is money.,Время — деньги.,general,time is money
368110,Declaration on Social and Legal Principles Con...,"Декларация о социальных и правовых принципах, ...",legal_docs,on the level
368121,A woman on the beach is jumping in the air und...,Женщина на пляже прыгает в воздухе под воздушн...,general,on the jump
368165,A pretty young lady talking on a cell phone wh...,Симпатичная молодая девушка разговаривает по м...,general,sitting pretty


In [ ]:
kazparc.dropna().to_csv('/content/drive/MyDrive/data/idioms/kazparc_processed.csv')

# Corpus cleaning

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/data/idioms/corpus_new_v.xlsx')
df.head()

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
0,Life is not a bed of roses.,Жизнь прожить — не поле перейти.,general,a bed of roses,Proverb,NaN,NaN,NaN,NaN,NaN
1,A bird in hand is worth two in the bush..,Кобчик в руках лучше сокола в небе.,general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
2,A bird in hand is worth two in the bush..,"Кобчик, что в руках, лучше сокола, что в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
3,A bird in the hand is worth two in the bush.,"Лучше воробей в руке, чем петух на кровле.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
4,A bird in the hand is worth two in the bush.,"Лучше синица в руках, чем журавль в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN


In [ ]:
df

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
0,Life is not a bed of roses.,Жизнь прожить — не поле перейти.,general,a bed of roses,Proverb,NaN,NaN,NaN,NaN,NaN
1,A bird in hand is worth two in the bush..,Кобчик в руках лучше сокола в небе.,general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
2,A bird in hand is worth two in the bush..,"Кобчик, что в руках, лучше сокола, что в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
3,A bird in the hand is worth two in the bush.,"Лучше воробей в руке, чем петух на кровле.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
4,A bird in the hand is worth two in the bush.,"Лучше синица в руках, чем журавль в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9016,Because of the structure and facilities at thi...,"Судя по структуре и возможностям этого клуба, ...",mass_media,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9017,Declaration on Social and Legal Principles Con...,"Декларация о социальных и правовых принципах, ...",legal_docs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9018,A woman on the beach is jumping in the air und...,Женщина на пляже прыгает в воздухе под воздушн...,general,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9019,A pretty young lady talking on a cell phone wh...,Симпатичная молодая девушка разговаривает по м...,general,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.dropna(subset='idiom_en')
df

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
0,Life is not a bed of roses.,Жизнь прожить — не поле перейти.,general,a bed of roses,Proverb,NaN,NaN,NaN,NaN,NaN
1,A bird in hand is worth two in the bush..,Кобчик в руках лучше сокола в небе.,general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
2,A bird in hand is worth two in the bush..,"Кобчик, что в руках, лучше сокола, что в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
3,A bird in the hand is worth two in the bush.,"Лучше воробей в руке, чем петух на кровле.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
4,A bird in the hand is worth two in the bush.,"Лучше синица в руках, чем журавль в небе.",general,a bird in the hand,Proverb,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1586,You reap what you sow.,"Что посеешь, то и пожнёшь.",general,you reap what you sow,Ambiguous,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True
1587,They give and reap what they sow.,"Дающий получит, посеявший пожнет.",general,you reap what you sow,True,NaN,NaN,NaN,1.0,False
1588,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ...",fiction,you reap what you sow,True,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True
1589,He reaps what he sows.,"Он пожнёт то, что и посеял.",general,you reap what you sow,True,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True


In [ ]:
df = df.loc[df['is_idiomatic'] != 'Proverb'].copy()
df

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
7,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
8,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
9,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
10,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
11,A mother sheep with two black baby sheeps.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1586,You reap what you sow.,"Что посеешь, то и пожнёшь.",general,you reap what you sow,Ambiguous,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True
1587,They give and reap what they sow.,"Дающий получит, посеявший пожнет.",general,you reap what you sow,True,NaN,NaN,NaN,1.0,False
1588,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ...",fiction,you reap what you sow,True,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True
1589,He reaps what he sows.,"Он пожнёт то, что и посеял.",general,you reap what you sow,True,"что посеешь, то и пожнёшь",NaN,NaN,1.0,True


In [ ]:
df['is_idiomatic'] = df['is_idiomatic'].replace({'True': True, 'False': False})
df['translation_equivalence'] = df['translation_equivalence'].replace({'True': True, 'False': False})
df.head()

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
7,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,DET + ADJ + NOUN,NaN,NaN,NaN
8,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,DET + ADJ + NOUN,NaN,NaN,NaN
9,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,DET + ADJ + NOUN,NaN,NaN,NaN
10,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,DET + ADJ + NOUN,NaN,NaN,NaN
11,A mother sheep with two black baby sheeps.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,DET + ADJ + NOUN,NaN,NaN,NaN


# Idiom structures

In [ ]:
!python -m spacy download en_core_web_lg
!python -m spacy download ru_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.4/513.4 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 23.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from tqdm import tqdm
import numpy as np

tqdm.pandas()
nlp_en = spacy.load("en_core_web_lg")
nlp_ru = spacy.load("ru_core_news_lg")

In [ ]:
from spacy import displacy

context = 'The children are playing in the field near the sheep.'
idiom = 'play the field'
displacy.render(nlp_en(context), jupyter=True, options={'compact': True})

In [ ]:
displacy.render(nlp_en(idiom), jupyter=True, options={'compact': True})

In [ ]:
context2 = 'All of them, along with a dozen more representatives of the local fauna, are listed in the Red Book of Kazakhstan'
idiom2 = 'in the red'
displacy.render(nlp_en(context2), jupyter=True, options={'compact': True})

In [ ]:
displacy.render(nlp_en(idiom2), jupyter=True, options={'compact': True})

In [ ]:
# idiom structures extraction

structures = {'en': [], 'ru': []}
for i, data in tqdm(df.iterrows(), total=df.shape[0]):
    if data['is_idiomatic'] is False:
        structures['en'].append(np.nan)
        structures['ru'].append(np.nan)
        continue

    structure_en = []
    structure_ru = []

    for token in nlp_en(data['idiom_en']):
        if token.dep_ == 'ROOT':
            structure_en.append(f'<{token.pos_}>')
            continue
        structure_en.append(token.pos_)
    structures['en'].append(' + '.join(structure_en))

    if data['idiom_ru'] is np.nan:
        structures['ru'].append(np.nan)
        continue
    for token in nlp_ru(data['idiom_ru']):
        if token.dep_ == 'ROOT':
            structure_ru.append(f'<{token.pos_}>')
            continue
        structure_ru.append(token.pos_)
    structures['ru'].append(' + '.join(structure_ru))

df['en_idiom_structure'] = structures['en']
df['ru_idiom_structure'] = structures['ru']

100%|██████████| 1540/1540 [00:13<00:00, 115.13it/s]


In [ ]:
df.head()

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
7,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
8,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
9,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
10,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
11,A mother sheep with two black baby sheeps.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.sample(10)

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
35,Even if there is some utility to our learning ...,Даже если существует какая-то польза от овладе...,fiction,a fact of life,False,NaN,NaN,NaN,NaN,NaN
1578,The people at the meeting came from all walks ...,На встрече присутствовали люди из самых разных...,general,walk of life,True,слой общества,<VERB> + ADP + NOUN,<NOUN> + NOUN,2.0,Partial
159,"Don't follow what you say, follow what you say...","Слову не верь, холсту подаренному верь: от сло...",general,be a picture,False,NaN,NaN,NaN,NaN,NaN
321,"As a populist, Trump has exploited the justifi...","Будучи популистом, Трамп эксплуатирует вполне ...",mass_media,downwardly mobile,True,NaN,ADV + <ADJ>,NaN,4.0,NaN
458,"When he went west, he wanted to be remembered.","Умирая, он хотел, чтобы его запомнили.",general,go west,True,NaN,<VERB> + NOUN,NaN,3.0,False
1470,"From time immemorial, our Kazakh ancestors con...",Наши предки-казахи испокон веков считали год к...,mass_media,time immemorial,True,испокон веков,NOUN + <ADJ>,ADV + <NOUN>,1.0,Partial
686,"But if I have a habit of showing discourtesy, ...","Но если мне свойственно проявлять неуважение, ...",fiction,little tin god,True,NaN,ADJ + NOUN + <PROPN>,NaN,3.0,False
109,You're in your 20s now. It's time to stop bein...,Тебе уже за 20. Пора прекратить строить из себ...,general,angry young man,True,NaN,ADJ + ADJ + <NOUN>,NaN,4.0,NaN
884,Drinks are on the house tonight!,Сегодня выпивка за счёт заведения!,general,on the house,True,NaN,<ADP> + DET + NOUN,NaN,4.0,NaN
107,"An eye for an eye, and a tooth for a tooth.","Око за око, зуб за зуб.",general,an eye for an eye and a tooth for a tooth,True,"око за око, зуб за зуб",DET + <NOUN> + ADP + DET + NOUN + CCONJ + DET ...,<NOUN> + ADP + NOUN + PUNCT + NOUN + ADP + NOUN,1.0,True


In [ ]:
df

,context_en,context_ru,domain,idiom_en,is_idiomatic,idiom_ru,en_idiom_structure,ru_idiom_structure,interlanguage_equivalence,translation_equivalence
7,A herd of black sheep grazing near the shore,Стадо черных овец пасется у берега,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
8,a lady bending over a rail looking at a white ...,"дама, склонившаяся над перилами, смотрит на бе...",general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
9,A sheep standing next to a baby black sheep.,Овца стоит рядом с чёрным ягнёнком.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
10,A large white sheep and two baby black sheep i...,Большая белая овца и две черные овечки в сарае.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
11,A mother sheep with two black baby sheeps.,Мать-овца с двумя черными овцами.,general,a black sheep,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1586,You reap what you sow.,"Что посеешь, то и пожнёшь.",general,you reap what you sow,Ambiguous,"что посеешь, то и пожнёшь",PRON + <VERB> + PRON + PRON + VERB,PRON + <VERB> + PUNCT + SCONJ + PART + <NOUN>,1.0,True
1587,They give and reap what they sow.,"Дающий получит, посеявший пожнет.",general,you reap what you sow,True,NaN,PRON + <VERB> + PRON + PRON + VERB,NaN,1.0,False
1588,"What you sow, you reap — no one will be given ...","Что посеешь, то и пожнёшь — никому не отдадут ...",fiction,you reap what you sow,True,"что посеешь, то и пожнёшь",PRON + <VERB> + PRON + PRON + VERB,PRON + <VERB> + PUNCT + SCONJ + PART + <NOUN>,1.0,True
1589,He reaps what he sows.,"Он пожнёт то, что и посеял.",general,you reap what you sow,True,"что посеешь, то и пожнёшь",PRON + <VERB> + PRON + PRON + VERB,PRON + <VERB> + PUNCT + SCONJ + PART + <NOUN>,1.0,True


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.to_excel('/content/drive/MyDrive/data/idioms/corpus_complete.xlsx', index=False)